# Criminalità Italia
Analisi concentrata su numero dei delitti in Italia

## Fonte Dati
   1. Istat
       - http://dati.istat.it/Index.aspx
       - http://jsonstatpy.readthedocs.io/en/latest/notebooks/istat_house_price_index.html
   2. ...


In [154]:
# Import
import os
import pandas as pd
# pip install jsonstat.py --> Per analizzare dati istat e altri https://json-stat.org/
# http://jsonstatpy.readthedocs.io/en/latest/index.html
import istat

In [155]:
# Imposto cache dir
cache_dir = os.path.abspath(os.path.join("..", "tmp", "istat_cached"))
istat.cache_dir(cache_dir)
print("cache_dir is '{}'".format(istat.cache_dir()))

cache_dir is '/home/acesarini/opendata/tmp/istat_cached'


In [156]:
# Lista Aree - http://dati.istat.it/Index.aspx
# istat.areas()

In [157]:
# AREA Giustizia e Sicurezza
istat_area_lab = istat.area('JUS')
# istat_area_lab

In [168]:
# DATASET delitti http://dati.istat.it/Index.aspx?DataSetCode=dccv_delittips
istat_dataset_taxdisoccu = istat_area_lab.dataset('DCCV_DELITTIPS')
istat_dataset_taxdisoccu

nr,name,nr. values,values (first 3 values)
0,Territory,148,"1:'Italy', 3:'Nord', 4:'Nord-ovest' ..."
1,Data type,3,"1:'number of crimes reported by the police forces to the judicial authority', 2:'percentage of the reference year crimes with alleged known offenders on crimes perpetrated in the reference year', 3:'crimes reported by the police forces to the judicial authority (per 100,000 population)' ..."
2,Type of crime,56,"1:'mass murder', 2:'intentional homicides', 3:'homicides for theft or robbery' ..."
3,Known offender identity,2,"2:'yes', 3:'total'"
4,Reference period of crime,3,"1:'before the reference year', 2:'during the reference year', 3:'before or during the reference year' ..."
5,Year,9,"2129:'2012', 2034:'2007', 2148:'2013' ..."


In [159]:
# Lista di tutte le Dimensioni con Valori -> Necessari per creare il dataset via le spec
# istat_dataset_taxdisoccu.dimensions()

In [170]:
# Es. Numero crimini registrati dalla polizia(Data Type) per Anno 2014 (Year) su tutto il Territorio Italia (Territory), Omicidi Intenzionali (Type of crime)
spec = {
    "Data type":1,                   # 1 -> number of crimes reported by the police forces to the judicial authority
    "Year":2167,                     # 2167 -> 2014
    #"Territory":,                   # 0 -> ALL
    #"Type of crime":,               # 0 -> ALL   
    "Known offender identity":3,     # 3 -> Total
    "Reference period of crime":2    # 2 -> during the reference year
}

collection = istat_dataset_taxdisoccu.getvalues(spec)

In [164]:
# Lista dei dataset presenti nella collection
# collection

In [171]:
# Jsonstat - Dataset
ds = collection.dataset(0)
ds

pos,id,label,size,role
0,IDITTER107,Territory,133,
1,IDREATI_PS,Type of crime,56,


In [172]:
# Trasformo dataset in Dataframe
df = ds.to_data_frame('Territory')

In [174]:
# Numero di delitti sul Territorio italiano, suddivisi per categoria (tot. 56 categorie)
df.head(56)

,Type of crime,Value
Territory,,
Italy,mass murder,20.0
Italy,intentional homicides,475.0
Italy,homicides for theft or robbery,27.0
Italy,homicides of mafia,45.0
Italy,terrorist homicides,NaN
Italy,attempted homicides,1250.0
Italy,infanticides,3.0
Italy,manslaughter,34.0
Italy,unintentional homicides,1633.0
